In [1]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
import re
%matplotlib inline

In [2]:
def read_tps(filename):
    with open(filename, 'r') as fp:
        contents = fp.read()
        
    print(contents)
    tps_match = re.search('tps = (\d+(?:\.\d+)) \(including connections establishing\)', contents)
    print(tps_match.groups)
    return float(tps_match.group(1))


In [3]:
import pathlib
from collections import defaultdict

param_regex = re.compile(r'(.*)-(\d+)-(simple|extended|prepared).results')
data_points = defaultdict(list)
plots = set([])

for filename in pathlib.Path('../results/').glob('*.results'):
    version, threads, query_mode = param_regex.match(str(filename)).groups()
    tps = read_tps(filename)
    plots.add(query_mode)
    data_points[(version, query_mode)].append((int(threads), tps))
    
if len(data_points) == 0:
    raise Exception('no data')
for value in data_points.values():
    value.sort()

t = np.arange(0.01, 20.0, 0.01)

plt.figure(figsize=(10, 10))

for i, query_mode in enumerate(plots):
    plt.subplot(3,1,i+1)
    plt.tight_layout()
    
    plt.title("Query mode {}".format(query_mode))
    for version, points in [(key[0], points) for key, points in data_points.items()
                            if key[1] == query_mode]:
        plt.xscale('log', basex=2)
        plt.plot(*zip(*points), label=version)

    plt.legend(loc='upper left')
    plt.xlabel('Number of threads')
    plt.ylabel('Transactions per second')

os.makedirs('figures/', exist_ok=True)
plt.savefig('figures/pgbench-comparison.pdf', bbox_inches='tight')
plt.show()

<Figure size 720x720 with 0 Axes>